# Внутренние испытания 20 июня 2020
После съемок

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import re
import os
import requests

In [2]:
URL = 'http://192.168.88.155:8081/logs'
def show_files():
    return(requests.get(URL).json())
def download(filename):
    r = requests.get(URL + '/' + filename)
    with open(filename, 'wb') as f:
        f.write(r.content)

In [3]:
def generate_dataframes_and_comments_from_file(filename):
    dt = {'time' : [], 'ph': []}
    dt_comments = {'time' : [], 'comment': []}
    with open(filename) as fp:
        line = fp.readline()
        last_value = 0
        while line:
            m = re.search(r'\[(\d{2}\:\d{2}\:\d{2})\]\:\s(\d+)$', line)
            if m:
                last_value = m.group(2)
                dt['time'].append(m.group(1))
                dt['ph'].append(last_value)
            else:
                m = re.search(r'\[(\d{2}\:\d{2}\:\d{2})\]\:\s(.+)$', line)
                if m:
                    time = m.group(1)
                    text = m.group(2)
                    if time in dt_comments['time']:
                        pass
                        """
                        for i in range(len(dt_comments['time'])):
                            if dt_comments['time'][i] == time:
                                dt_comments['comment'][i] += ", " + text
                        """
                    else:
                        dt_comments['time'].append(time)
                        dt_comments['comment'].append(text)
                    if time not in dt['time']:
                        dt['time'].append(time)
                        dt['ph'].append(last_value)
            line = fp.readline()
    df = pd.DataFrame(data = dt)
    return(df, dt_comments)

In [4]:
def draw_file(filename):
    df, comments = generate_dataframes_and_comments_from_file(filename)
    fig = go.Figure([go.Scatter(x=df['time'], y=df['ph'])])
    for i in range(len(comments['time'])):    
        fig.add_annotation(go.layout.Annotation(
            x = comments['time'][i],
            text = comments['comment'][i],
            align='center',
            showarrow=False,
            yanchor='bottom',
            textangle=-90))
    fig.show()

In [14]:
show_files()

['sequence.py-2020.06.20_16:35:11.log', 'latest.log']

16:35 Откалибровано. Первый запуск на растворах NaOH и кислоты. vb1=vb2=1.65В (default)

16:43 vb1=vb2=0.5В - график сместился по y вниз, амплитуда та же

16:47 вернул vb1=vb2=1.65В

16:50 ADC range 3V

17:00 ADC range 1.2V

17:05 ADC range 1.1V

17:07 vb1=vb2=0.5В

17:08 вернул vb1=vb2=1.65В





In [28]:
#download('sequence.py-2020.06.20_16:35:11.log')
draw_file('sequence.py-2020.06.20_16:35:11.log')

17:34 снова откалибровал

ADC 1.10V

vb1 = vb2 = 1.65V

18:25 обнаружил, что давление на нуле. Накачал.

In [40]:
download('sequence.py-2020.06.20_17:35:37.log')
draw_file('sequence.py-2020.06.20_17:35:37.log')

In [30]:
show_files()

['calibrate.py-2020.06.20_17:34:49.log',
 'sequence.py-2020.06.20_16:35:11.log',
 'sequence.py-2020.06.20_17:35:37.log',
 'latest.log']